In [5]:
import glob, os
import pandas as pd

In [7]:
study_cases = ["outputs"]
file_name = "gen_cap.csv"

In [8]:
frames = []
for path in study_cases:
    df = pd.read_csv(os.path.join(path,file_name))
    df['scenario'] = path
    frames.append(df)

In [19]:
frames[0][frames[0]["gen_tech"] == "WIND"]

,GENERATION_PROJECT,PERIOD,gen_tech,gen_load_zone,gen_energy_source,GenCapacity,SuspendGen_total,GenStorageCapacity,GenCapitalRecovery,GenFixedOMCosts,scenario
120,Alena,2024,WIND,Bio Bio,Eolica,67.09,0,.,0.0,0.0,outputs
121,Alena,2026,WIND,Bio Bio,Eolica,67.09,0,.,0.0,0.0,outputs
122,Alena,2029,WIND,Bio Bio,Eolica,67.09,0,.,0.0,0.0,outputs
123,Alena,2030,WIND,Bio Bio,Eolica,67.09,0,.,0.0,0.0,outputs
124,Alena,2031,WIND,Bio Bio,Eolica,67.09,0,.,0.0,0.0,outputs
...,...,...,...,...,...,...,...,...,...,...,...
11899,VALLEDELOSVIENTOS,2030,WIND,Antofagasta_5,Eolica,90.00,0,.,0.0,0.0,outputs
11900,VALLEDELOSVIENTOS,2031,WIND,Antofagasta_5,Eolica,90.00,0,.,0.0,0.0,outputs
11901,VALLEDELOSVIENTOS,2033,WIND,Antofagasta_5,Eolica,90.00,0,.,0.0,0.0,outputs
11902,VALLEDELOSVIENTOS,2040,WIND,Antofagasta_5,Eolica,90.00,0,.,0.0,0.0,outputs


In [22]:
# concatenate along vertical axis, replacing missing values with 0
df = pd.concat(frames, axis=0, sort=False)
# default group
df['technology'] = 'Térmico'
# list of search key and technology labels
updates = [
    ('PV', 'Solar'),
    ('WIND', 'Eólico'),
    ('TES', 'TES'),
    ('Bomb', 'Central de Bombeo'),
    ('HYDRO', 'Hidroeléctrica'),
    ('CSP-TES', 'CSP-TES'),
    ('CAES', 'CAES'),
    ('ESS', 'ESS')
]
for search, tech in updates:
    df.loc[df['gen_tech'] == search, 'technology'] = tech

In [24]:
# aggregate GenCapacity values
agg = (
    df.groupby(['scenario', 'PERIOD', 'technology'])
    ['GenCapacity']
    .sum()
    .unstack(-1)   # create columns for each technology
    .loc[:, [              # reorder columns
        'Térmico',
        'Central de Bombeo',
        'Hidroeléctrica',
        'TES',
        'CSP-TES',
        'CAES',
        'ESS',
        'Solar',
        'Eólico'
    ]]
)

# add dummy rows for each scenario to make gaps when plotting later
agg = agg.reset_index()
dummies = pd.DataFrame({'scenario': agg['scenario'].unique()})
agg = pd.concat([agg,dummies], sort=False, axis=0)
agg = agg.sort_values(['scenario', 'PERIOD'])

In [27]:
# remove scenario names except for first period,
# to improve Excel category labeling
agg.loc[agg['PERIOD'] != agg['PERIOD'].min(), 'scenario'] = ''

# change the dummy scenario names to ' ' to create gaps in Excel
agg.loc[agg['PERIOD'].isnull(), 'scenario'] = ' '

outfile = os.path.join('outputs', 'capacity_by_tech_by_scenario.xlsx')
agg.to_excel(outfile, index=False)
print("saved {}.".format(outfile))

saved outputs\capacity_by_tech_by_scenario.xlsx.
